# Dependências

In [5]:
# Para cálculo do hash
import hashlib

# Para deduplicação
import os
import shutil

# Cálculo do hash de um arquivo

In [6]:
def calcular_hash(arquivo, algoritmo='sha256'):
    """
    Calcula o hash de um arquivo.

    Parâmetros:
    arquivo (str): O caminho do arquivo para calcular o hash.
    algoritmo (str): O algoritmo de hash a ser utilizado (padrão: 'sha256').

    Retorna:
    str: O hash hexadecimal do arquivo.
    """
    hasher = hashlib.new(algoritmo)
    with open(arquivo, 'rb') as f:
        while True:
            bloco = f.read(4096)
            if not bloco:
                break
            hasher.update(bloco)
    return hasher.hexdigest()

# Verificação da integridade de um arquivo

In [7]:
def verificar_integridade(arquivo, hash_esperado, algoritmo='sha256'):
    """
    Verifica se a integridade de um arquivo é mantida, comparando o hash calculado com o hash esperado.

    Parâmetros:
    arquivo (str): O caminho do arquivo para verificar a integridade.
    hash_esperado (str): O hash esperado do arquivo.
    algoritmo (str): O algoritmo de hash a ser utilizado (padrão: 'sha256').

    Retorna:
    bool: True se a integridade for mantida, False caso contrário.
    """
    hash_calculado = calcular_hash(arquivo, algoritmo)
    return hash_calculado == hash_esperado


# Deduplicação de arquivos

In [8]:
def deduplicar_arquivos(diretorio):
    """
    Deduplica arquivos em um diretório com base no hash.

    Parâmetros:
    diretorio (str): O diretório onde a deduplicação será realizada.
    """
    hashes = {}
    duplicatas = []

    # Itera sobre os arquivos no diretório
    for raiz, _, arquivos in os.walk(diretorio):
        for arquivo in arquivos:
            caminho_arquivo = os.path.join(raiz, arquivo)
            hash_arquivo = calcular_hash(caminho_arquivo)

            # Verifica se o hash já existe
            if hash_arquivo in hashes:
                duplicatas.append((caminho_arquivo, hashes[hash_arquivo]))
            else:
                hashes[hash_arquivo] = caminho_arquivo

    # Move os arquivos duplicados para uma pasta de duplicatas
    pasta_duplicatas = os.path.join(diretorio, 'Duplicatas')
    if not os.path.exists(pasta_duplicatas):
        os.makedirs(pasta_duplicatas)

    for duplicata, original in duplicatas:
        novo_caminho = os.path.join(pasta_duplicatas, os.path.basename(duplicata))
        shutil.move(duplicata, novo_caminho)
        print(f"Arquivo '{os.path.basename(duplicata)}' duplicado. Movido para {pasta_duplicatas}.")

# Detecção de intrusão ou vírus

In [9]:
def verificar_integridade_arquivos(lista_hashes, diretorio):
    """
    Verifica a integridade dos arquivos em relação aos hashes previamente armazenados.

    Parâmetros:
    lista_hashes (dict): Um dicionário com os hashes dos arquivos.
    diretorio (str): O diretório onde os arquivos estão localizados.
    """
    for arquivo, hash_original in lista_hashes.items():
        caminho_arquivo = os.path.join(diretorio, arquivo)
        if os.path.isfile(caminho_arquivo):
            if verificar_integridade(caminho_arquivo, hash_original):
                print(f"O arquivo '{arquivo}' permanece íntegro.")
            else:
                print(f"O arquivo '{arquivo}' foi modificado!")
        else:
            print(f"O arquivo '{arquivo}' não foi encontrado no diretório.")

In [10]:
def criar_lista_hashes(diretorio):
    """
    Cria uma lista com os hashes dos arquivos em uma determinada pasta.

    Parâmetros:
    diretorio (str): O diretório onde os arquivos estão localizados.

    Retorna:
    dict: Um dicionário onde as chaves são os nomes dos arquivos e os valores são os hashes correspondentes.
    """
    lista_hashes = {}
    for raiz, _, arquivos in os.walk(diretorio):
        for arquivo in arquivos:
            caminho_arquivo = os.path.join(raiz, arquivo)
            hash_arquivo = calcular_hash(caminho_arquivo)
            lista_hashes[arquivo] = hash_arquivo
    return lista_hashes


# Execução

### Integridade

In [26]:
arquivo = 'arquivo.txt'
hash_esperado = calcular_hash(arquivo)

In [27]:
if verificar_integridade(arquivo, hash_esperado):
    print("Integridade do arquivo verificada com sucesso!")
else:
    print("Integridade do arquivo comprometida.")

Integridade do arquivo verificada com sucesso!


In [28]:
hash_esperado = 'hashAlterado123'

if verificar_integridade(arquivo, hash_esperado):
    print("Integridade do arquivo verificada com sucesso!")
else:
    print("Integridade do arquivo comprometida.")

Integridade do arquivo comprometida.


### Deduplicação

In [28]:
# Deduplica arquivos em um diretório chamado Deduplicação
diretorio_deduplicacao = 'Deduplicação'
deduplicar_arquivos(diretorio_deduplicacao)

Arquivo 'Duplicado.txt' duplicado. Movido para Deduplicação\Duplicatas.
Arquivo 'Duplicata.txt' duplicado. Movido para Deduplicação\Duplicatas.


### Detecção de intrusão ou vírus

In [16]:
diretorio_arquivos = 'Detecção de intrusão'
lista_hashes = criar_lista_hashes(diretorio_arquivos)
lista_hashes

{'Arquivo 1.txt': 'c7a4909a3b7e7567533605ee12b7cc28ef1b480f4ad6ce78239d832c5ef39aef',
 'Arquivo 2.txt': 'bb68fb3f32f412318e39cd5b38c77fb37aa17b9ea4c833543b4f7b0acf50d33d',
 'Arquivo 3.txt': '81f926663d2e05d101f663cc7d9417299b59be56a774c8f8e38fc8400cb85ae2'}

In [15]:
# Verificar integridade dos arquivos em relação aos hashes previamente armazenados
verificar_integridade_arquivos(lista_hashes, diretorio_arquivos)

O arquivo 'Arquivo 1.txt' permanece íntegro.
O arquivo 'Arquivo 2.txt' permanece íntegro.
O arquivo 'Arquivo 3.txt' permanece íntegro.


In [23]:
lista_hashes['Arquivo 1.txt'] = 'hashAlterado123'
lista_hashes

{'Arquivo 1.txt': 'hashAlterado123',
 'Arquivo 2.txt': 'bb68fb3f32f412318e39cd5b38c77fb37aa17b9ea4c833543b4f7b0acf50d33d',
 'Arquivo 3.txt': '81f926663d2e05d101f663cc7d9417299b59be56a774c8f8e38fc8400cb85ae2'}

In [24]:
verificar_integridade_arquivos(lista_hashes, diretorio_arquivos)

O arquivo 'Arquivo 1.txt' foi modificado!
O arquivo 'Arquivo 2.txt' permanece íntegro.
O arquivo 'Arquivo 3.txt' permanece íntegro.
